# Longest Palindromic Substring

**Disclaimer: This contains a solution to a LeetCode problem. Anyone who wishes to work out these problems on their own should stop reading now.**

Given a string `s`, find the longest palindromic substring in `s`. You may assume that the maximum length of `s` is 1000.

Example:
```
Input: "babad"

Output: "bab"

Note: "aba" is also a valid answer.
```

Example:
```
Input: "cbbd"

Output: "bb"
```

*Note: A palindrome is just a string that reads the same from right to left as left to right.*

### Strategy: Brute Force
1. Create `isPalindrome` helper function.
2. Attempt to bruce force check all starting and ending positions.

In [1]:
# Brute Force
# Time: O(n^3)
# Space: O(1)

class Solution:
    def isPalindrome(self, s):
        """ (str) -> bool
        
        Returns whether a string is a palindrome
        
        >>> isPalindrome('aba')
        True
        """
        
        return str(s) == str(s)[::-1]
        
    
    def longestPalindrome(self, s):
        """ (str) -> str
        
        Return longest palindrome in a string.
        
        >>> longestPalindrome("babad")
        "bab"
        >>> longestPalindrome("cbbd")
        "bb"
        """
        
        # Return error for edge cases
        if s is None or not isinstance(s, str):
            return "Invalid input."
        
        # Initialize length of string
        n = len(s)
        
        # Initalize max_len
        max_len = 1
        best_string = ''
        
        # Iterate through string and check
        for start in range(n):
            for end in range(start, n):
                if self.isPalindrome(s[start:end+1]):
                    if len(s[start:end+1]) > max_len:
                        best_string = s[start:end+1]
                        max_len = len(s[start:end+1])
        
        if max_len == 1:
            return s[0]
        
        return best_string

In [2]:
%%timeit -n1000 -r6
assert Solution().isPalindrome("abcdefghijklmnopponmlkjihgfedcba")
assert Solution().longestPalindrome(123) == "Invalid input."
assert Solution().longestPalindrome("babad") == "bab"
assert Solution().longestPalindrome("cbbd") == "bb"
assert Solution().longestPalindrome("abcdefghijjiklm") == "ijji"
assert Solution().longestPalindrome("abcdefghijjiklmmlkz") == "klmmlk"
assert Solution().longestPalindrome("abcdefghijklmnopqrstuvwkyz123456789") == "a"
assert Solution().longestPalindrome("abcdefghijklmnopqrstuvwkyz1234567899") == "99"
assert Solution().longestPalindrome("abcdefghijklmnopponmlkjihgfedcba") == "abcdefghijklmnopponmlkjihgfedcba"

1.53 ms ± 82.1 µs per loop (mean ± std. dev. of 6 runs, 1000 loops each)


### Strategy: Dynamic Programming

Dynamic programmingis a method for solving a complex problem by breaking it down into a collection of simpler subproblems, solving each of those subproblems just once, and storing their solutions. The next time the same subproblem occurs, instead of recomputing its solution, one simply looks up the previously computed solution.

For this case, we can maintain a `table[i][j]` that is filled from bottem up, ie 1 length palindromes, followed by 2 length, etc. Since each new larger palindrome has to be centered around a smaller palindrome, we only need to check if the end case characters are the same.
1. Create table.
2. Populate all 1 length substrings as palindromes.
3. Populate all 2 length substrings, `s[i] == s[i+1]` then palindrome.
4. Populate for 3 and up, if `table[i+1][j-1]` is True and `s[i] == s[j]` then palindrome.
5. Keep track of max length palindrome and return.

In [3]:
# Dynamic Programming
# Time: O(n^2)
# Space: O(n^2)

class Solution:
    def longestPalindrome(self, s):
        """ (str) -> str
        
        Return longest palindrome in a string.
        
        >>> longestPalindrome("babad")
        "bab"
        >>> longestPalindrome("cbbd")
        "bb"
        """
        
        # Return error for edge cases
        if s is None or not isinstance(s, str):
            return "Invalid input."
        
        # Initialize length of string
        n = len(s)
        
        # Create hash table to store results
        # Table[i][j] false if substring s[i:j] not palindrome
        table = [[False for x in range(n)] for y in range(n)] 
        
        # Base case: strings length one are palindromes
        max_len = 1
        i = 0
        while i < n:
            table[i][i] = True
            i += 1
            
        # Check 2-length substrings
        start = 0
        i = 0
        while i < n - 1:
            if s[i] == s[i+1]:
                table[i][i+1] = True
                start = i
                max_len = 2
            i += 1
            
        # Check k-length substrings starting at 3
        k = 3
        
        while k <= n:
            # Get starting index
            i = 0
            while i < n - k + 1:
                # Get ending index for length k
                j = i + k - 1
                
                # Check if lower length substring already a palindrome and end letters match
                if table[i+1][j-1] is True and s[i] == s[j]:
                    # Update table with true
                    table[i][j] = True
                    
                    # If length exceeds max, update search
                    if k > max_len:
                        start = i
                        max_len = k
                
                # Move starting position
                i += 1
            # Increase length of search   
            k += 1
            
        return s[start: start + max_len]

In [4]:
%%timeit -n1000 -r6
assert Solution().longestPalindrome(123) == "Invalid input."
assert Solution().longestPalindrome("babad") == "bab"
assert Solution().longestPalindrome("cbbd") == "bb"
assert Solution().longestPalindrome("abcdefghijjiklm") == "ijji"
assert Solution().longestPalindrome("abcdefghijjiklmmlkz") == "klmmlk"
assert Solution().longestPalindrome("abcdefghijklmnopqrstuvwkyz123456789") == "a"
assert Solution().longestPalindrome("abcdefghijklmnopqrstuvwkyz1234567899") == "99"
assert Solution().longestPalindrome("abcdefghijklmnopponmlkjihgfedcba") == "abcdefghijklmnopponmlkjihgfedcba"

749 µs ± 26.8 µs per loop (mean ± std. dev. of 6 runs, 1000 loops each)


Our dynamic solution finishes in about half the time.

### Strategy: Optimizing Dynamic Programming

We can come at the problem from a different angle. Say we have a base string 'aab' with a max length palindrome of 2. If we add another character, perhaps 'aaba', our max length palindrome increased by 1 to 3. If we another letter, 'aabaa', our max length palindrome increased by 2 to 5.

In fact, adding another character to a string can only result in a max length increase of 1 or 2 and it has to include the added letter. Knowing this we can simplify our search:
1. Keep track of current max_len palindrome.
2. Interate through adding character `i` to end.
3. For each added character, check if substrings ending in that character with length max_len + 1 or max_len + 2 are palindromes.
4. Update max_len and repeat.

In [5]:
# Manacher's Algorithm
# Time: O(n^2)
# Space: O(n)

class Solution:
    def isPalindrome(self, s):
        """ (str) -> bool
        
        Returns whether a string is a palindrome
        
        >>> isPalindrome('aba')
        True
        """
        
        return str(s) == str(s)[::-1]
    
    
    def longestPalindrome(self, s):
        """ (str) -> str
        
        Return longest palindrome in a string.
        
        >>> longestPalindrome("babad")
        "bab"
        >>> longestPalindrome("cbbd")
        "bb"
        """
        # Return error for edge cases
        if s is None or not isinstance(s, str):
            return "Invalid input."
        
        # Return string if already palindrome
        if self.isPalindrome(s):
            return s
        
        # Initate starting variables
        n = len(s)
        max_len = 1
        start = 0
        
        # Iterate through starting index
        for i in range(n):
            # Search until Palindrome is found, 
            if i - max_len >= 1 and self.isPalindrome(s[i-max_len-1: i+1]):
                start = i - max_len - 1
                max_len += 2
                continue

            if i - max_len >= 0 and self.isPalindrome(s[i-max_len: i+1]):
                start = i - max_len
                max_len += 1
                
        return s[start:start+max_len]

In [6]:
%%timeit -n1000 -r6
assert Solution().longestPalindrome(123) == "Invalid input."
assert Solution().longestPalindrome("babad") == "bab"
assert Solution().longestPalindrome("cbbd") == "bb"
assert Solution().longestPalindrome("abcdefghijjiklm") == "ijji"
assert Solution().longestPalindrome("abcdefghijjiklmmlkz") == "klmmlk"
assert Solution().longestPalindrome("abcdefghijklmnopqrstuvwkyz123456789") == "a"
assert Solution().longestPalindrome("abcdefghijklmnopqrstuvwkyz1234567899") == "99"
assert Solution().longestPalindrome("abcdefghijklmnopponmlkjihgfedcba") == "abcdefghijklmnopponmlkjihgfedcba"

185 µs ± 29.3 µs per loop (mean ± std. dev. of 6 runs, 1000 loops each)


Just by playing around with our search space, we reduced our run time by nearly 10x the brute force method.